## Import packages

In [1]:
import cv2
from keras.utils import load_img
from keras.saving import load_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from segment_anything import sam_model_registry, SamPredictor
from skimage.measure import regionprops, regionprops_table
from tqdm import trange, tqdm

import segmenteverygrain as seg

%matplotlib qt

## Load models

In [19]:
# Load UNET model
unet = load_model(
    "./models/seg_model.keras",
    custom_objects={"weighted_crossentropy": seg.weighted_crossentropy},
)

# Download SAM model (only downloads it if it does not exist)
import os 
if not os.path.exists("./models/sam_vit_h_4b8939.pth"):
    import urllib.request
    url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
    urllib.request.urlretrieve(url, "./models/sam_vit_h_4b8939.pth")

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images. Images with ~2000 pixels along their largest dimension are a good start and allow the user to get an idea about how well the segmentation works.

If you have a much larger image, see the section **"Run segmentation on large image"** at the end of the notebook. Running the `predict_large_image` function takes a lot longer (e.g., several hours), but it is possible to analyze very large images with tens of thousands of grains.

Image used below is available from [here](https://github.com/zsylvester/segmenteverygrain/blob/main/torrey_pines_beach_image.jpeg).

In [14]:
# replace this with the path to your image:
fname = "./examples/barton_creek/barton_creek_image.jpg"
image = np.array(load_img(fname))
image_pred = seg.predict_image(image, unet, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts
# (and longer processing times):
labels, coords = seg.label_grains(image, image_pred, dbs_max_dist=20.0)

segmenting image tiles...


100%|█████████████████████████████████████████████| 8/8 [00:02<00:00,  3.80it/s]


creating masks using SAM...


100%|███████████████████████████████████████| 1120/1120 [01:04<00:00, 17.29it/s]


finding overlapping polygons...


...

finding best polygons...


100%|█████████████████████████████████████████| 309/309 [00:09<00:00, 31.79it/s]


creating labeled image...


100%|████████████████████████████████████████| 336/336 [00:01<00:00, 209.99it/s]


CPU times: user 2min 36s, sys: 48.4 s, total: 3min 24s
Wall time: 1min 41s


Use the figure created in the next cell to check the quality of the Unet labeling (sometimes it doesn't work at all) and the distribution of SAM prompts (= black dots). If the Unet prediction is of poor quality, it is a good idea to create some training data and fine tune the base model so that it works better with the images of interest.

In [4]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.imshow(image_pred)
plt.scatter(np.array(coords)[:, 0], np.array(coords)[:, 1], c="k")
plt.xticks([])
plt.yticks([]);

In [5]:
# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(
    sam,
    image,
    image_pred,
    coords,
    labels,
    min_area=400.0,
    plot_image=True,
    remove_edge_grains=False,
    remove_large_objects=False,
)

creating masks using SAM...


100%|███████████████████████████████████████| 1120/1120 [01:02<00:00, 17.98it/s]


finding overlapping polygons...


1112it [00:07, 152.39it/s]


finding best polygons...


100%|█████████████████████████████████████████| 309/309 [00:09<00:00, 31.69it/s]


creating labeled image...


100%|████████████████████████████████████████| 336/336 [00:01<00:00, 213.44it/s]


## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [6]:
grain_inds = []
cid1 = fig.canvas.mpl_connect(
    "button_press_event",
    lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax),
)
cid2 = fig.canvas.mpl_connect(
    "key_press_event",
    lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax),
)

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [7]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'labels' array after deleting and merging grains (the 'all_grains' list is updated when doing the deletion and merging):

In [8]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 334/334 [00:00<00:00, 2349.30it/s]
334it [00:00, 1250.07it/s]


Plot the updated set of grains:

In [9]:
fig, ax = plt.subplots(figsize=(15, 10))
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap="Paired", plot_image=True)
seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0]);

100%|████████████████████████████████████████| 334/334 [00:01<00:00, 212.47it/s]


## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [10]:
predictor = SamPredictor(sam)
predictor.set_image(image)  # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect(
    "button_press_event", lambda event: seg.onclick(event, ax, coords, image, predictor)
)
cid4 = fig.canvas.mpl_connect(
    "key_press_event", lambda event: seg.onpress(event, ax, fig)
)

In [11]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [12]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|███████████████████████████████████████| 334/334 [00:00<00:00, 2416.37it/s]
334it [00:00, 1242.03it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [49]:
cid5 = fig.canvas.mpl_connect(
    "button_press_event", lambda event: seg.click_for_scale(event, ax)
)

number of pixels: 374.26


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [50]:
n_of_units = 10.0  # centimeters if using 'barton_creek_image.jpg'
units_per_pixel = n_of_units / 374.26  # length of scale bar in pixels

In [51]:
props = regionprops_table(
    labels.astype("int"),
    intensity_image=image,
    properties=(
        "label",
        "area",
        "centroid",
        "major_axis_length",
        "minor_axis_length",
        "orientation",
        "perimeter",
        "max_intensity",
        "mean_intensity",
        "min_intensity",
    ),
)
grain_data = pd.DataFrame(props)
grain_data["major_axis_length"] = (
    grain_data["major_axis_length"].values * units_per_pixel
)
grain_data["minor_axis_length"] = (
    grain_data["minor_axis_length"].values * units_per_pixel
)
grain_data["perimeter"] = grain_data["perimeter"].values * units_per_pixel
grain_data["area"] = grain_data["area"].values * units_per_pixel**2

In [52]:
grain_data.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,1.638460,43.381699,1309.247495,2.283864,1.114569,1.515264,6.622803,71.0,74.0,81.0,22.122876,22.969499,28.977778,0.0,0.0,0.0
1,2,0.789602,80.790235,445.920434,1.928161,0.590840,0.257955,4.501364,211.0,213.0,212.0,80.692586,82.019892,84.280289,0.0,0.0,2.0
2,3,0.680371,211.855194,1287.741868,1.435305,0.719128,-0.258269,3.775356,68.0,69.0,71.0,19.193075,19.867786,21.357817,0.0,0.0,0.0
3,4,0.793886,303.482014,1784.839029,1.477209,0.807340,0.185842,4.243338,191.0,190.0,188.0,116.599820,113.858813,111.160072,2.0,0.0,5.0
4,5,0.591131,314.114734,1820.256039,1.203063,0.672594,1.374467,3.277642,147.0,142.0,138.0,41.028986,40.020531,39.427536,3.0,0.0,0.0


In [53]:
grain_data.to_csv(fname[:-4] + ".csv")  # save grain data to CSV file

In [54]:
# plot histogram of grain axis lengths
# note that input data needs to be in milimeters!
# these limits are for 'barton_creek_image.jpg'
fig, ax = seg.plot_histogram_of_axis_lengths(
    grain_data["major_axis_length"] * 10,
    grain_data["minor_axis_length"] * 10,
    binsize=0.4,
    xlimits=[2, 128],
)

## Save mask and grain labels to PNG files

In [55]:
dirname = "./examples/barton_creek/"
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split("/")[-1][:-4] + "_mask.png", mask_all)
# Save the image as a PNG file
cv2.imwrite(
    dirname + fname.split("/")[-1][:-4] + "_image.png",
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
)

True

## Run segmentation on large image
In this case 'fname' points to an image that is larger than a few megapixels and has thousands of grains.
The 'predict_large_image' function breaks the input image into smaller patches and it runs the segmentation process on each patch.

The image used below (from [Mair et al., 2022, Earth Surface Dynamics](https://esurf.copernicus.org/articles/10/953/2022/)) is available [here](https://github.com/zsylvester/segmenteverygrain/blob/main/mair_et_al_L2_DJI_0382_image.jpg).

In [16]:
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # needed if working with very large images
fname = "./examples/mair_et_al_L2_DJI_0382/mair_et_al_L2_DJI_0382_image.jpg"
all_grains, image_pred, all_coords = seg.predict_large_image(
    fname, unet, sam, min_area=400.0, patch_size=2000, overlap=200
)

segmenting image tiles...


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.59it/s]


creating masks using SAM...


100%|███████████████████████████████████████| 2804/2804 [03:05<00:00, 15.12it/s]


finding overlapping polygons...


...

finding best polygons...


100%|█████████████████████████████████████████| 519/519 [00:10<00:00, 47.65it/s]


creating labeled image...
processed patch #6 out of 6 patches


4753it [00:02, 2320.82it/s]
100%|█████████████████████████████████████████| 331/331 [00:54<00:00,  6.11it/s]

CPU times: user 30min 20s, sys: 8min 36s, total: 38min 56s
Wall time: 20min 37s


In [57]:
# plot results
image = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15, 10))
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(image, all_grains, ax, cmap="Paired")
plt.axis("equal")
plt.xlim([0, np.shape(image)[1]])
plt.ylim([np.shape(image)[0], 0]);

100%|██████████████████████████████████████| 4399/4399 [00:13<00:00, 322.04it/s]


In [24]:
# this is a faster way of deleting false positives (because it avoids highlighting and deleting the 'bad' grains)
grain_inds = []
cid1 = fig.canvas.mpl_connect(
    "button_press_event",
    lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax, select_only=True),
)

In [25]:
# delete polygons from 'all_grains'
grain_inds = np.unique(grain_inds)
grain_inds = sorted(grain_inds, reverse=True)
for ind in tqdm(grain_inds):
    all_grains.remove(all_grains[ind])

0it [00:00, ?it/s]


After plotting the results, you will want to use the functions for deleting, merging, and adding grains (see above), before saving the results (same workflow as for a small image).

In [58]:
all_grains, labels, mask_all = seg.get_grains_from_patches(ax, image)

100%|██████████████████████████████████████| 4399/4399 [00:07<00:00, 567.78it/s]
4399it [00:01, 2800.52it/s]


In [59]:
dirname = "./examples/mair_et_al_L2_DJI_0382/"
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split("/")[-1][:-4] + "_mask.png", mask_all)
# Save the image as a PNG file
cv2.imwrite(
    dirname + fname.split("/")[-1][:-4] + "_image.png",
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
)

True

### Finetuning the base model

In [ ]:
# patchify images and masks
input_dir = "./examples/unet_training/Masks_and_images/"  # the input directory should contain files with 'image' and 'mask' in their filenames
patch_dir = (
    "./examples/unet_training/"  # a directory called "Patches" will be created here
)
image_dir, mask_dir = seg.patchify_training_data(input_dir, patch_dir)

In [ ]:
# create training, validation, and test datasets
train_dataset, val_dataset, test_dataset = seg.create_train_val_test_data(
    image_dir, mask_dir, augmentation=True
)

In [ ]:
# load base model weights and train the model with the new data
model = seg.create_and_train_model(
    train_dataset,
    val_dataset,
    test_dataset,
    model_file="./models/seg_model.keras",
    epochs=100,
)

In [ ]:
# save finetuned model as new model (this then can be loaded using "model = load_model("new_model.keras", custom_objects={'weighted_crossentropy': seg.weighted_crossentropy})"
model.save("./examples/unet_training/new_model.keras")